In [1]:
from elasticsearch import Elasticsearch
import json

client = Elasticsearch('localhost:9200')

response = client.search(
    index = "facebook_nested",
    doc_type  = "fanpage",
    body = {
        "size": 0,
        "query": {
            "match_all": {
            }
        },
        "aggs": {
            "users_per_month": {
                "terms": {
                    "size": 0,
                    "field": "name"
                }
            }
        }
    }
)

# print (response)
# I choose "HTC", "ASUS Taiwan" and "Acer Taiwan"
print (json.dumps(response, ensure_ascii=False , indent = 2))
# print (response["aggregations"]["twitter_accounts_in_range"]["value"])

{
  "timed_out": false,
  "hits": {
    "hits": [],
    "max_score": 0.0,
    "total": 162
  },
  "aggregations": {
    "users_per_month": {
      "sum_other_doc_count": 0,
      "doc_count_error_upper_bound": 0,
      "buckets": [
        {
          "key": "85度C",
          "doc_count": 1
        },
        {
          "key": "ASUS Taiwan",
          "doc_count": 1
        },
        {
          "key": "Acer Taiwan",
          "doc_count": 1
        },
        {
          "key": "Airwaves 精神啦啦隊",
          "doc_count": 1
        },
        {
          "key": "American Express",
          "doc_count": 1
        },
        {
          "key": "BMW Taiwan總代理 汎德",
          "doc_count": 1
        },
        {
          "key": "Bayer",
          "doc_count": 1
        },
        {
          "key": "BlackBerry",
          "doc_count": 1
        },
        {
          "key": "CDC",
          "doc_count": 1
        },
        {
          "key": "China Airlines 中華航空",
          "doc_count": 1


In [75]:
from elasticsearch import Elasticsearch
from bokeh.charts import Donut, reset_output, output_notebook, show
# from bokeh.io import output_notebook
import json, math
client = Elasticsearch('localhost:9200')

response = client.search(
    index = "facebook_nested",
    doc_type  = "post",
    body = {
      "query": {
        "terms": {
          "fname": ["HTC", "ASUS Taiwan", "Acer Taiwan"]
        }
      },
      "size": 0,
      "aggs": {
        "fname": {
          "terms": {
            "field": "fname"
          },
          "aggs": {
            "shares_count": {
              "terms": {
                "field": "shares.count",
                "size": 0
              }
            }
          }
        }
      }
    }
)

for fanpage in response['aggregations']['fname']['buckets']:
    total_shares = 0
    for shares in fanpage['shares_count']['buckets']:
        total_shares += shares["doc_count"]
    print(fanpage["key"] + " total posts:" + str(fanpage["doc_count"]))
    print(fanpage["key"] + " total shares " + str(total_shares))
    print(fanpage["key"] + " post share rate = " + str(total_shares/fanpage["doc_count"]))
    print()
    withoutshare = fanpage["doc_count"] - total_shares
    data = dict(label=["Share  " + str(round(total_shares / fanpage["doc_count"] * 100, 2)) + "%"
                       , "No share  " + str(round(withoutshare / fanpage["doc_count"] * 100, 2)) + "%"]
                , value=[total_shares, withoutshare])
    print(data)
    reset_output()
    output_notebook()
    show(Donut(data, label='label', values='value', title=fanpage["key"]))
    
# print (response)
# I choose "HTC", "ASUS Taiwan" and "Acer Taiwan"
# print (json.dumps(response, ensure_ascii=False , indent = 2))


ASUS Taiwan total posts:2248
ASUS Taiwan total shares 2160
ASUS Taiwan post share rate = 0.9608540925266904

{'value': [2160, 88], 'label': ['Share  96.09%', 'No share  3.91%']}


Loading BokehJS ...

HTC total posts:1069
HTC total shares 1068
HTC post share rate = 0.9990645463049579

{'value': [1068, 1], 'label': ['Share  99.91%', 'No share  0.09%']}


Loading BokehJS ...

Acer Taiwan total posts:453
Acer Taiwan total shares 401
Acer Taiwan post share rate = 0.8852097130242825

{'value': [401, 52], 'label': ['Share  88.52%', 'No share  11.48%']}


Loading BokehJS ...

In [63]:
from elasticsearch import Elasticsearch
import json
import pandas as pd
from bokeh.charts import BoxPlot, reset_output, output_notebook, show
from pandas import DataFrame

client = Elasticsearch('localhost:9200')

response = client.search(
    index = "facebook_nested",
    doc_type  = "comment",
    body = {
      "query": {
        "terms": {
          "fname": ["HTC", "ASUS Taiwan", "Acer Taiwan"]
        }
      },
      "size": 0,
      "aggs": {
        "fname": {
          "terms": {
            "field": "fname",
            "size": 0
          },
          "aggs": {
            "comments_per_month": {
              "date_histogram": {
                "field": "created_time",
                "interval": "month",
                "format": "yyyy-MM"
              }
            }
          }
        }
      }
    }
)

# print (response)
# I choose "HTC", "ASUS Taiwan" and "Acer Taiwan"

df = []
for fanpage in response['aggregations']['fname']['buckets']:
    for comments_per_month in fanpage['comments_per_month']['buckets']:
        row = {}
        row['fanpage_name'] = fanpage['key']
        row['date'] = comments_per_month['key_as_string']
        row['comments_per_month'] = comments_per_month['doc_count']
        df.append(row)

df = pd.DataFrame(df)
df.groupby('fanpage_name')['comments_per_month'].agg({'comments_per_month(min)': min,
                                                      'comments_per_month(max)': max,
                                                      'comments_per_month(mean)': 'mean'})
# print (json.dumps(response, ensure_ascii=False , indent = 2))


,comments_per_month(mean),comments_per_month(min),comments_per_month(max)
fanpage_name,,,
ASUS Taiwan,3739.390244,254,13590
Acer Taiwan,297.585366,2,3656
HTC,5344.439024,562,11723


In [81]:
from elasticsearch import Elasticsearch
import json
import pandas as pd
from bokeh.charts import BoxPlot, reset_output, output_notebook, show
from pandas import DataFrame

client = Elasticsearch('localhost:9200')

response = client.search(
    index = "facebook_nested",
    doc_type  = "comment",
    body = {
      "query": {
        "terms": {
          "fname": ["HTC", "ASUS Taiwan", "Acer Taiwan"]
        }
      },
      "size": 0,
      "aggs": {
        "fname": {
          "terms": {
            "field": "fname",
            "size": 0
          },
          "aggs": {
            "comments_per_month": {
              "date_histogram": {
                "field": "created_time",
                "interval": "month",
                "format": "yyyy-MM"
              }
            }
          }
        }
      }
    }
)

# print (response)
# I choose "HTC", "ASUS Taiwan" and "Acer Taiwan"

print("Fanpage comments per month")

for fanpage in response['aggregations']['fname']['buckets']:
    df = []
    for comments_per_month in fanpage['comments_per_month']['buckets']:
        row = {}
        row['fanpage_name'] = fanpage['key']
        row['date'] = comments_per_month['key_as_string']
        row['comments_per_month'] = comments_per_month['doc_count']
        df.append(row)
    p = BoxPlot(df, values='comments_per_month', label='date', title=fanpage["key"], color="white", legend=False, plot_width=800)
    reset_output()
    output_notebook()
    show(p, notebook_handle=True)

print("Total comments per month")

df = []
for fanpage in response['aggregations']['fname']['buckets']:
    for comments_per_month in fanpage['comments_per_month']['buckets']:
        row = {}
        row['fanpage_name'] = fanpage['key']
        row['date'] = comments_per_month['key_as_string']
        row['comments_per_month'] = comments_per_month['doc_count']
        df.append(row)
p = BoxPlot(df, values='comments_per_month', label='date', title="All", color="white", legend=False, plot_width=800)
reset_output()
output_notebook()
show(p, notebook_handle=True)
# print (json.dumps(response, ensure_ascii=False , indent = 2))


Fanpage comments per month


Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Total comments per month


Loading BokehJS ...

In [76]:
from elasticsearch import Elasticsearch
import json
import pandas as pd
client = Elasticsearch('localhost:9200')

response = client.search(
    index = "facebook_nested",
    doc_type  = "comment",
    body = {
      "query": {
        "terms": {
          "fname": ["HTC", "ASUS Taiwan", "Acer Taiwan"]
        }
      },
      "size": 0,
      "aggs": {
        "fname": {
          "terms": {
            "field": "fname",
            "size": 0
          },
          "aggs": {
            "from_id": {
              "terms": {
                "field": "from.id",
                "size": 1
              }
            }
          }
        }
      }
    }
)

# print (response)
# I choose "HTC", "ASUS Taiwan" and "Acer Taiwan"

for fanpage in response['aggregations']['fname']['buckets']:
    for users in fanpage['from_id']['buckets']:
        print(fanpage["key"] + " total unique user comments " + str(users["doc_count"]))
        print()


HTC total unique user comments 27039

ASUS Taiwan total unique user comments 3271

Acer Taiwan total unique user comments 119

